In [1]:
!pip3 install lxml
#import all tools for project
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import numpy as np
import datetime
import matplotlib.pyplot as plt
import re
from time import sleep

# Determining the Frequency of Gendered Language

### References
Scikit-learn: Machine Learning in Python, Pedregosa et al., JMLR 12, pp. 2825-2830, 2011.

Dr. Mattei, Tulane University. SimpleText Notebook. 2020. 
Bo Pang and Lillian Lee, A Sentimental Education: Sentiment Analysis Using Subjectivity Summarization Based on Minimum Cuts, Proceedings of ACL 2004.

In [2]:
#Import CSV containing mass amount of scraped reviews for analysis
reviewsByGender = pd.read_csv("./revs_gendered.csv")
reviewsByGender

,Unnamed: 0,Review,Gender
0,0,Very good,Male
1,1,"Exelent he really care about you, he is very p...",Male
2,2,Dr. A-Rahim was very knowledgeable about my lo...,Male
3,3,Horrible physician treats you like a kid. Pts ...,Female
4,4,Dr Aaberg has been a very good dr for my husba...,Male
...,...,...,...
8890,8890,Very helpful and sincere about my pain needs! ...,Male
8891,8891,"Several of my children see Dr. Zach, she is a ...",Female
8892,8892,Amazing Pediatric Neurologist stays up to date...,Female
8893,8893,Dr. Zach has been a Godsend for our daughter! ...,Female


In [3]:
# Vectorize the whole thing...
import sklearn
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


# Vectorize and play with token sizes...
vec = TfidfVectorizer(min_df = 2, 
                      max_df = 0.98, 
                      ngram_range=(1,1)) # play with min_df and max_df

reviews = reviewsByGender['Review']

# transform this into a sparse vector!
vec.fit(reviews)
tf_idf_sparse = vec.transform(reviews)
tf_idf_sparse

#only 222 unique words due to min_df 0.03 --> change to 2, maybe change to 5 (could get meaningless words)

<8895x7365 sparse matrix of type '<class 'numpy.float64'>'
	with 322903 stored elements in Compressed Sparse Row format>

In [4]:
revs = dict(zip(reviewsByGender.Review, reviewsByGender.Gender))

In [5]:
#get list of cleaned, stemmed adjectives, for later analysis. 
vect = CountVectorizer()
vect.fit(revs)
words = vect.get_feature_names()

adjectives = []
stop_words = list(set(stopwords.words('english')))

# remove stopwords 
stopped = [w for w in words if not w in stop_words]
    
# parts of speech tagging for each word 
pos = nltk.pos_tag(stopped)

# make a list of  all adjectives identified by the allowed word types list above
for w in pos:
    if w[1][0] == 'J':
        adjectives.append(w[0].lower())

In [6]:
print(tf_idf_sparse[0, :])

  (0, 7004)	0.5113918885739566
  (0, 2832)	0.8593476225025365


In [7]:
# We can now use this to classify the reviews!! but we need to test/train split again.

# Split..
X_train, X_test, y_train, y_test = train_test_split(tf_idf_sparse, 
                                                    reviewsByGender['Gender'], 
                                                    test_size=0.2)

In [8]:
logisticRegr = LogisticRegression(max_iter=100000, class_weight='balanced') 
model = logisticRegr.fit(X_train, y_train)

In [9]:
logisticRegr.classes_

array(['Female', 'Male'], dtype=object)

In [10]:
logisticRegr.coef_[:,0]

array([0.17571364])

In [11]:
# Make a dataframe with the words, coefficients, and classes...
recs = []

for w,i in vec.vocabulary_.items():
    recs.append([str(w)] + list(logisticRegr.coef_[:,i]))
# If we only have one class then we only get weight..
# df_weights = pd.DataFrame(tripples, columns=['word']+list(logisticRegr.classes_))
df_weights = pd.DataFrame(recs, columns=['word', 'weight'])

In [12]:
df_weights.sort_values('weight', ascending=False)[:25]

,word,weight
3,he,13.331486
85,his,8.833455
84,him,7.214020
328,surgery,3.497624
920,man,1.771411
86,outstanding,1.528613
984,informative,1.421560
1358,mother,1.376369
546,knee,1.329925
14,was,1.309487


In [13]:
#print only adjectives
#male adjectives
maleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=False)[:25]
maleAdjectives

,word,weight
86,outstanding,1.528613
1111,arrogant,1.117337
1,good,1.102414
4753,faber,1.055406
21,great,0.849152
2291,kindness,0.815535
723,fine,0.721397
110,much,0.680306
1815,pa,0.649378
7156,udall,0.625234


In [14]:
df_weights.sort_values('weight', ascending=True)[:25]

,word,weight
44,she,-13.941201
396,her,-10.924884
4699,ma,-2.729389
4442,eady,-1.732895
5981,kabbash,-1.526565
6504,nabors,-1.484614
6708,pabolu,-1.468074
303,unprofessional,-1.423117
588,woman,-1.374594
7309,yablonski,-1.363092


In [15]:
#female adjectives
femaleAdjectives = df_weights[df_weights['word'].isin(adjectives)].sort_values('weight', ascending=True)[:25]
#maybe take out adjective specification
#vocabulary being pruned down too aggressively
femaleAdjectives

,word,weight
4442,eady,-1.732895
303,unprofessional,-1.423117
1452,sweet,-1.180887
712,happy,-1.040171
3512,cabalona,-0.951560
15,knowledgeable,-0.942386
481,wonderful,-0.875642
999,ok,-0.772957
1314,annual,-0.756692
1533,perfect,-0.754531


In [16]:
#verify results
agentic_terms = ['assertive','confident','aggressive','ambitious','dominan','forceful','independent','daring','outspoken','intellectua','earn','gain','know','insight','think']
communal_terms = ['sympathetic','kind','help','affection','sensitive','nurtur','agreeab','tactful','interpersonal','warm','car','tactful']
socio_communal = ['husband','wife','kid','babies','brother','child','colleague','family']

print('Results for most commonly found female adjectives:')
print('Agentic: ', femaleAdjectives['word'].isin(agentic_terms).sum())
print('Communal: ', femaleAdjectives['word'].isin(communal_terms).sum())
print('Socio-communal: ', femaleAdjectives['word'].isin(socio_communal).sum())
print('')
print('Results for most commonly found male adjectives:')
print('Agentic: ', maleAdjectives['word'].isin(agentic_terms).sum())
print('Communal: ', maleAdjectives['word'].isin(communal_terms).sum())
print('Socio-communal: ', maleAdjectives['word'].isin(socio_communal).sum())

Results for most commonly found female adjectives:
Agentic:  0
Communal:  0
Socio-communal:  0

Results for most commonly found male adjectives:
Agentic:  0
Communal:  0
Socio-communal:  0


In [17]:
#can use medical reviews to run with recommnedation letter reviews to predict which is which, then throw out 
#heavily medical terms

# recommendation system
# maybe seek out positive words that don't distinguish between male and female
# 4-class problem rather than two-class problem: take in star data and add pos/neg dimension to the data
# word vector stuff from NLP class: use language models and figure out how often it appears in male context vs
# female context. Ex: look at similarities between words: compare "leadership" to "man/men" 

#doubt raisers may not come out from this analysis: "she may not be the best in her class..."
# subtle, hard to catch -> maybe the word "not"
# append "not" to nearest adjective, negated adjectives might indicate doubt raisers
# hedging terms to weaken a statement (weasel words): refusing to take a stand/hesitant
# "somewhat", "mostly"

# how to rephrase it: female-correlated to neutral, remove hedging terms
# take an education based approach: "this can be conveyed as ___. If that is not your intention, consider ___"
# crowdsourcing: emphasize anonymization